In [ ]:
encoder_url="https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
preprocess_url="https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

In [ ]:
!pip install --upgrade tensorflow_hub
import tensorflow as tf
import tensorflow_hub as hub


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!pip3 install tensorflow_text
!python -m pip install tensorflow_text
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 61.2 MB/s eta 0:00:00


In [ ]:
##load the dataset
import io
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving Salman_Khan_VS_Sharuk_Khan - Sheet2.csv to Salman_Khan_VS_Sharuk_Khan - Sheet2.csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded.get('Salman_Khan_VS_Sharuk_Khan - Sheet2.csv')))
df.head()

,Name,Comment,Time,Likes,Reply Count,Supporter
0,Obed Khan,Shah Rukh Khan sirf ek,2023-01-30T02:44:56Z,0,0,Shahrukh khan
1,Aasif Patel,Salman khan,2023-01-29T13:49:48Z,0,0,Salman Khan
2,V Shejule Status,Only Salman bhai<br>Bhot Hard Bhai✌,2023-01-28T17:53:55Z,0,0,Salman Khan
3,Avinash Tudu rushika,Bollywood ka sabse bada super star srk,2023-01-28T17:38:39Z,0,0,Shahrukh khan
4,DexterDrubo Animations-,In India Salman Khan has bigger stardom than S...,2023-01-28T11:34:45Z,0,0,Shahrukh khan


In [ ]:

df['Supporter'].unique()



array(['Shahrukh  khan', 'Salman Khan', 'Both', 'comment', 'Comment',
       'Sharukh Khan', 'Shahrukh Khan', ' Shahrukh Khan', 'sharukh Khan',
       nan, 'comments', 'both', 'Ssha', 'cooment', 'SSA', 'commets'],
      dtype=object)

In [ ]:
df['Supporter']=df['Supporter'].str.lower()
df['Supporter'].unique()

array(['shahrukh  khan', 'salman khan', 'both', 'comment', 'sharukh khan',
       'shahrukh khan', ' shahrukh khan', nan, 'comments', 'ssha',
       'cooment', 'ssa', 'commets'], dtype=object)

In [ ]:
df['Supporter'].value_counts()

salman khan       461
sharukh khan      386
comment            62
shahrukh  khan     61
both               20
shahrukh khan       2
comments            2
 shahrukh khan      1
ssha                1
cooment             1
ssa                 1
commets             1
Name: Supporter, dtype: int64

In [ ]:
ind1=df[df['Supporter'].str.contains("sharukh khan", na=False)].index
ind2=df[df['Supporter'].str.contains("shahrukh  khan", na=False)].index
ind3=df[df['Supporter'].str.contains(" shahrukh khan", na=False)].index


In [ ]:
df.loc[ind1,'Supporter']="shahrukh khan" 
df.loc[ind2,'Supporter']="shahrukh khan"
df.loc[ind3,'Supporter']="shahrukh khan" 

In [ ]:
df['Supporter'].unique()

array(['shahrukh khan', 'salman khan', 'both', 'comment', nan, 'comments',
       'ssha', 'cooment', 'ssa', 'commets'], dtype=object)

In [ ]:
search=["salman khan","shahrukh khan"]
drop_ind=df[~df["Supporter"].str.contains('|'.join(search),na=False)].index

In [ ]:
df.drop(drop_ind, inplace=True)

In [ ]:
df['Supporter'].unique()

array(['shahrukh khan', 'salman khan'], dtype=object)

In [ ]:
import numpy as np
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Supporter=df['Supporter']
Supporter=np.array(Supporter).reshape(-1,1)


df['encoded_labels']=''
enc=le.fit_transform(Supporter)

df['encoded_labels']=enc
df.head()

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,Name,Comment,Time,Likes,Reply Count,Supporter,encoded_labels
0,Obed Khan,Shah Rukh Khan sirf ek,2023-01-30T02:44:56Z,0,0,shahrukh khan,1
1,Aasif Patel,Salman khan,2023-01-29T13:49:48Z,0,0,salman khan,0
2,V Shejule Status,Only Salman bhai<br>Bhot Hard Bhai✌,2023-01-28T17:53:55Z,0,0,salman khan,0
3,Avinash Tudu rushika,Bollywood ka sabse bada super star srk,2023-01-28T17:38:39Z,0,0,shahrukh khan,1
4,DexterDrubo Animations-,In India Salman Khan has bigger stardom than S...,2023-01-28T11:34:45Z,0,0,shahrukh khan,1


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Comment'],df["encoded_labels"])


In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
#Intialize bert layer this convert text into vector
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='Comment')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
#Intialize the neural network

#This layer will be used to prevent model overfitting
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
#Dense layer only has one neuron. We also initialize the activation function as sigmoid. sigmoid is used when we have output values that between 0 and 1.
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

In [ ]:
#We also name the layer as output because this is our output layer. Lets now add the input and output layers to construct the final model as shown below 
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Comment (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['Comment[0][0]']                
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [ ]:
model.fit(X_train,y_train, epochs=10)

Epoch 1/10
22/22 [==============================] - 317s 14s/step - loss: 0.7658 - accuracy: 0.5271 - precision: 0.5157 - recall: 0.6369
Epoch 2/10
22/22 [==============================] - 304s 14s/step - loss: 0.6816 - accuracy: 0.5710 - precision: 0.5805 - recall: 0.4613
Epoch 3/10
22/22 [==============================] - 318s 14s/step - loss: 0.6281 - accuracy: 0.6413 - precision: 0.6842 - recall: 0.5030
Epoch 4/10
22/22 [==============================] - 303s 14s/step - loss: 0.6253 - accuracy: 0.6457 - precision: 0.6374 - recall: 0.6488
Epoch 5/10
22/22 [==============================] - 304s 14s/step - loss: 0.6046 - accuracy: 0.6603 - precision: 0.6486 - recall: 0.6756
Epoch 6/10
22/22 [==============================] - 304s 14s/step - loss: 0.5944 - accuracy: 0.6837 - precision: 0.6818 - recall: 0.6696
Epoch 7/10
22/22 [==============================] - 304s 14s/step - loss: 0.5783 - accuracy: 0.6881 - precision: 0.6752 - recall: 0.7054
Epoch 8/10
22/22 [=======================

In [ ]:
#Prediction
data=[input("Write your review about your favourite supporter\n")]

prediction=model.predict(data)
if prediction>=0.5:
    print("\nSalman Khan Supporter")
else:
    print("\nSharukh Khan Supporter")

Write your review about your favourite supporter
Salu is best
1/1 [==============================] - 0s 492ms/step

Salman Khan Supporter
